# Lesson 2 - Wrapping the QA Agent into an A2A Server

In this lesson, you will take the `PolicyAgent` created in Lesson 1 and wrap it into an Agent2Agent (A2A) server. This makes the agent discoverable and callable by other agents using the A2A protocol. You will define the agent's identity using an `AgentCard`, describe its capabilities using `AgentSkill`, and set up the request handling logic.


## 2.1. Define the A2A Server

The server code is provided in `a2a_policy_agent.py`. This script performs the following steps:
1.  **Imports**: Loads necessary modules including the A2A SDK (`a2a.server`).
2.  **Executor**: Defines `PolicyAgentExecutor`, which bridges the A2A request context to your `PolicyAgent` class.
3.  **Metadata**: configure the `AgentSkill` (defining *what* it can do) and the `AgentCard` (defining *who* it is, including its URL).
4.  **Application**: Creates and runs an `A2AStarletteApplication` using `uvicorn`.


In [1]:
from IPython.display import Code, display

display(Code("a2a_policy_agent.py"))

"""
A2A (Agent-To-Agent) Health Insurance Policy Agent
--------------------------------------------------
This file defines and runs an Agent using the A2A protocol.
As a Data Analyst, you can think of this as setting up a "service" or "API"
that other agents (or users) can talk to.

Here, we are wrapping a standard Python class (`PolicyAgent`, which contains our core logic/LLM calls)
into a web server that speaks the standard A2A language, allowing it to easily interact with other agents.
"""

import os

# uvicorn is a lightning-fast web server for Python. We use it to run our agent's API.
import uvicorn

# These are core A2A components that handle the server-side logic of an agent.
# Let's break down the role of each:
# 
# 1. AgentExecutor & RequestContext:
#    - AgentExecutor is a class you must inherit from to define how your agent handles incoming messages.
#    - RequestContext holds the details of the incoming request (who sent it, what the message is, etc.).
from a2a.server.agent_execution import AgentExecutor, RequestContext

# 2. A2AStarletteApplication:
#    - Starlette is a fast web framework (similar to Flask or FastAPI).
#    - This specific class essentially builds the "House" for your agent; it packages your
#      agent's logic, its description (AgentCard), and its routing into a web server application.
from a2a.server.apps import A2AStarletteApplication

# 3. EventQueue:
#    - When your agent finishes thinking and generates an answer, it doesn't just "return" it.
#    - It places the answer into an EventQueue. Imagine this as an "Outbox" where messages wait
#      to be delivered back over the network to whoever asked the question.
from a2a.server.events import EventQueue

# 4. DefaultRequestHandler:
#    - This is the "Bouncer" or "Receptionist" at the front door. It receives the raw HTTP web
#      requests, translates them, and decides what to do with them (like passing them to your AgentExecutor).
from a2a.server.request_handlers import DefaultRequestHandler

# 5. InMemoryTaskStore:
#    - When someone asks a question, it might take the AI a few seconds to answer.
#    - This store acts like a "clipboard" keeping track of ongoing conversations/tasks in the server's memory.
from a2a.server.tasks import InMemoryTaskStore

# These types help us define our agent's "business card" so other agents know what it can do.
from a2a.types import (
    AgentCapabilities,
    AgentCard,
    AgentSkill,
)

# A utility to create a text message in the format that A2A expects.
from a2a.utils import new_agent_text_message

# These are custom helpers for this specific project (loading environment variables, etc.)
from helpers import setup_env
# This is our core logic class. It likely contains the prompt engineering and LLM calls.
from policy_agent import PolicyAgent


class PolicyAgentExecutor(AgentExecutor):
    """
    The Executor is the bridge between the A2A server and your custom agent logic.
    Whenever this agent receives a request over the network, the A2A server will call
    the `execute` method of this class.
    """
    def __init__(self) -> None:
        # Initialize our custom logic (the brain of the agent)
        self.agent = PolicyAgent()

    async def execute(
        self,
        context: RequestContext,
        event_queue: EventQueue,
    ) -> None:
        """
        This method is triggered when a user or another agent sends a message to this agent.
        """
        # 1. Get the user's question from the request context
        prompt = context.get_user_input()
        
        # 2. Pass the question to our core policy agent to get an answer (e.g., using an LLM to read a policy DB)
        response = self.agent.answer_query(prompt)
        
        # 3. Format the answer into a standard A2A text message
        message = new_agent_text_message(response)
        
        # 4. Put the message into the event queue, which sends it back to the caller
        await event_queue.enqueue_event(message)

## 2.2. Run the Policy A2A Server

Now to activate your configured A2A agent, you would need to run your agent server. You can run the agent server using `uv`:

- Open a terminal as instructed below.
- Type `uv run a2a_policy_agent.py` to run the server and activate your A2A agent.

<div style="background-color:#e8f0fe; padding:15px; border-left:5px solid #4285f4; border-radius:4px">
    <b>Terminal Access:</b> Please open a new terminal window in your Jupyter environment to run the server.
    <br>You can typically do this by selecting <i>File -> New -> Terminal</i> from the menu.
</div>

## 2.3. Resources

- [Agent2Agent Protocol Documentation](https://a2a-protocol.org/)
- [A2A Python SDK Documentation](https://a2a-protocol.org/latest/sdk/python/api/)
- [Starlette Documentation](https://www.starlette.io/)